In [ ]:
import cv2
import numpy as np
from scipy import ndimage

In [ ]:
# 色彩空间：灰度、BGR、HSV
img = cv2.imread("XXX.jpg")
img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
cv2.imshow('1',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 傅里叶变换
# 使用np的fft包实现
# 原始图像由许多频率组成，分离这些频率来理解图像和提取roi
img = cv2.imread("XXX.jpg",0) # 以灰度读取
f = np.fft.fft2(img)
fshift = np.fft.fftshift(f)
magnitude_spectrum = 20 * np.log(np.abs(fshift))
row, cols = img.shape
crow, ccol = int(row / 2), int(cols / 2)
fshift[crow - 30: crow+30, ccol - 30: ccol + 30] = 0
f_ishift = np.fft.ifftshift(fshift)
img_back = np.fft.ifft2(f_ishift)
img_back = np.abs(img_back)
cv2.imshow('1',img)
cv2.imshow('2',magnitude_spectrum)
cv2.imshow('3',img_back)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 高通滤波器：检测某个区域，根据亮度差值来提升像素亮度
# 核是一组权重的组合，作用于一块区域并生成一个像素
# 权重加起来为1时，不会改变图像的亮度
# 权重加起来为0，会得到边缘检测核，把边缘转为白色
kernel_3x3 = np.array([[-1, -1, -1],
                   [-1,  8, -1],
                   [-1, -1, -1]])
kernel_5x5 = np.array([[-1, -1, -1, -1, -1],
                       [-1,  1,  2,  1, -1],
                       [-1,  2,  4,  2, -1],
                       [-1,  1,  2,  1, -1],
                       [-1, -1, -1, -1, -1]])
img = cv2.imread("XXX.jpg",0)
k3 = ndimage.convolve(img, kernel_3x3)
k5 = ndimage.convolve(img, kernel_5x5)
blurred = cv2.GaussianBlur(img, (17,17), 0)
g_hpf = img - blurred
cv2.imshow("3x3", k3)
cv2.imshow("5x5", k5)
cv2.imshow("g_hpf", g_hpf)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 低通滤波器：像素与周围亮度差值小于阈值时
# 平滑该像素亮度，用于去噪和模糊，如高斯模糊

In [ ]:
# 边缘检测
def strokeEdges(src, blurKsize = 3, edgeKsize = 5):
    """将图像转换成黑色边缘和白色背景，并归一化(0~1)
       乘以源图像将边缘变黑"""
    dst = src.copy()
    # 避免噪声干扰，先进行滤波模糊
    if blurKsize >= 3:
        blurredSrc = cv2.medianBlur(src, blurKsize)
        graySrc = cv2.cvtColor(blurredSrc, cv2.COLOR_BGR2GRAY)
    else:
        graySrc = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    # 然后需要转换为灰度空间
    # 通过 Laplacian、Sobel、Scharr等函数将非边缘转为黑色，边缘转为白色
    cv2.Laplacian(graySrc, cv2.CV_8U, graySrc, ksize = edgeKsize)
    normalizedInverseAlpha = (1.0 / 255) * (255 - graySrc)
    channels = cv2.split(src)
    for channel in channels:
        channel[:] = channel * normalizedInverseAlpha
    cv2.merge(channels, dst)
    return dst

img = cv2.imread("XXX.jpg")
img1 = strokeEdges(img)
cv2.imshow("img", img)
cv2.imshow("img1", img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 定制卷积核
# 一般卷积滤波
class VConvolutionFilter(object):
    """A filter that applies a convolution to V (or all of BGR)."""
    def __init__(self, kernel):
        self._kernel = kernel
    def apply(self, src, dst):
        """Apply the filter with a BGR or gray source/destination."""
        cv2.filter2D(src, -1, self._kernel, dst)
# 对彩色图像，filter2D会对每个通道用同样的函数
# 对单独通道时，使用split()和merge()函数

# 锐化滤波
class SharpenFilter(VConvolutionFilter):
    """A sharpen filter with a 1-pixel radius."""
    def __init__(self):
        kernel = numpy.array([[-1, -1, -1],
                              [-1,  9, -1],
                              [-1, -1, -1]])
        VConvolutionFilter.__init__(self, kernel)

# 边缘检测滤波器(权重和为0)
class FindEdgesFilter(VConvolutionFilter):
    """An edge-finding filter with a 1-pixel radius."""
    def __init__(self):
        kernel = numpy.array([[-1, -1, -1],
                              [-1,  8, -1],
                              [-1, -1, -1]])
        VConvolutionFilter.__init__(self, kernel)

# 平均模糊滤波器(权重和为1，近邻权重为正)   
class BlurFilter(VConvolutionFilter):
    """A blur filter with a 2-pixel radius."""
    def __init__(self):
        kernel = numpy.array([[0.04, 0.04, 0.04, 0.04, 0.04],
                              [0.04, 0.04, 0.04, 0.04, 0.04],
                              [0.04, 0.04, 0.04, 0.04, 0.04],
                              [0.04, 0.04, 0.04, 0.04, 0.04],
                              [0.04, 0.04, 0.04, 0.04, 0.04]])
        VConvolutionFilter.__init__(self, kernel)

# 脊状、浮雕效果
class EmbossFilter(VConvolutionFilter):
    """An emboss filter with a 1-pixel radius."""
    def __init__(self):
        kernel = numpy.array([[-2, -1, 0],
                              [-1,  1, 1],
                              [ 0,  1, 2]])
        VConvolutionFilter.__init__(self, kernel)

In [ ]:
# canny: 集成5步骤
# 高斯滤波去噪、计算梯度、NMS边缘、双阈值、连接性
img = cv2.imread("XXX.jpg", 0)
cv2.imshow("canny", cv2.Canny(img, 200, 300))
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
# 轮廓检测
# 创建黑色图像
img = np.zeros((200, 200), dtype=np.uint8)
# 放置白色方块
img[50:150, 50:150] = 255
# 二值化
ret, thresh = cv2.threshold(img, 127, 255, 0)
# 寻找轮廓：输入图像、层次方法, 轮廓逼近方法
# 输出：轮廓、层次（直接修改原图像）
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
img = cv2.drawContours(color, contours, -1, (0,255,0), 2)
cv2.imshow("contours", color)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 高级轮廓
# cv2.pyrDown从一个高分辨率大尺寸的图像向上构建一个金字塔（尺寸变小，分辨率降低）
img = cv2.pyrDown(cv2.imread("XXX.jpg", 
                             cv2.IMREAD_UNCHANGED))
ret, thresh = cv2.threshold(cv2.cvtColor(img.copy(), cv2.COLOR_BGR2GRAY) , 
                            127, 255, 
                            cv2.THRESH_BINARY)
contours, hier = cv2.findContours(thresh, 
                                  cv2.RETR_EXTERNAL, 
                                  cv2.CHAIN_APPROX_SIMPLE)
for c in contours:
  x,y,w,h = cv2.boundingRect(c) # 计算边界框
  cv2.rectangle(img, (x,y), (x+w, y+h), (0, 255, 0), 2)
  rect = cv2.minAreaRect(c)     # 包围目标的最小矩形区域
  box = cv2.boxPoints(rect)     # 计算顶点（浮点）
  box = np.int0(box)            # 转为整形
  # 检查轮廓的最小闭圆
  cv2.drawContours(img, [box], 0, (0,0, 255), 3)
  (x,y),radius = cv2.minEnclosingCircle(c)
  center = (int(x),int(y))
  radius = int(radius)
  img = cv2.circle(img,center,radius,(0,255,0),2)

cv2.drawContours(img, contours, -1, (255, 0, 0), 1)
cv2.imshow("contours", img)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
# 多边形
img = cv2.pyrDown(cv2.imread("XXX.jpg", cv2.IMREAD_UNCHANGED))
ret, thresh = cv2.threshold(cv2.cvtColor(img.copy(), 
                                         cv2.COLOR_BGR2GRAY) , 
                            127, 255, 
                            cv2.THRESH_BINARY)
black = cv2.cvtColor(np.zeros((img.shape[1], 
                               img.shape[0]), 
                              dtype=np.uint8), 
                     cv2.COLOR_GRAY2BGR)

contours, hier = cv2.findContours(thresh, 
                                  cv2.RETR_EXTERNAL, 
                                  cv2.CHAIN_APPROX_SIMPLE)
for cnt in contours:
  # 通过轮廓周长计算差值
    epsilon = 0.01 * cv2.arcLength(cnt,True)
    # cv2.approxPolyDP 计算近似的多边形框
    # 输入轮廓、差值、是否闭合
    # 差值越小，近似的越接近
    approx = cv2.approxPolyDP(cnt,epsilon,True)
    hull = cv2.convexHull(cnt)  # 获取凸包
    cv2.drawContours(black, [cnt], -1, (0, 255, 0), 2)
    cv2.drawContours(black, [approx], -1, (255, 255, 0), 2)
    cv2.drawContours(black, [hull], -1, (0, 0, 255), 2)


cv2.imshow("hull", black)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
# 直线检测
# HoughLines和HoughLinesP，后者使用概率霍夫变换，计算更快
img = cv2.imread('lines.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,50,120)
minLineLength = 20
maxLineGap = 5
# 接收单通道二值图像
lines = cv2.HoughLinesP(edges,1,np.pi/180,20,minLineLength,maxLineGap)
for x1,y1,x2,y2 in lines[0]:
  cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)
cv2.imshow("edges", edges)
cv2.imshow("lines", img)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
# 圆检测
planets = cv2.imread('XXX.jpg')
gray_img = cv2.cvtColor(planets, cv2.COLOR_BGR2GRAY)
img = cv2.medianBlur(gray_img, 5)
cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,120,
                            param1=100,param2=30,
                           minRadius=0,maxRadius=0)
circles = np.uint16(np.around(circles))
for i in circles[0,:]:
    # draw the outer circle
    cv2.circle(planets,(i[0],i[1]),i[2],(0,255,0),2)
    # draw the center of the circle
    cv2.circle(planets,(i[0],i[1]),2,(0,0,255),3)
cv2.imshow("HoughCirlces", planets)
cv2.waitKey()
cv2.destroyAllWindows()